<a href="https://colab.research.google.com/github/martin-fabbri/recsys-pipeline/blob/main/tiktok-pipeline/1_feature_backfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <span style='color:#ff5f27'> 📝 Imports

In [7]:
!rm *.py
!wget https://raw.githubusercontent.com/martin-fabbri/recsys-pipeline/refs/heads/main/tiktok-pipeline/features.py

!rm requirements.*
!wget https://raw.githubusercontent.com/martin-fabbri/recsys-pipeline/refs/heads/main/tiktok-pipeline/requirements.txt
!pip install -r requirements.txt --quiet

  and should_run_async(code)



--2024-12-08 18:42:13--  https://raw.githubusercontent.com/martin-fabbri/recsys-pipeline/refs/heads/main/tiktok-pipeline/features.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5649 (5.5K) [text/plain]
Saving to: ‘features.py’

features.py         100%[===================>]   5.52K  --.-KB/s    in 0s      

2024-12-08 18:42:14 (55.7 MB/s) - ‘features.py’ saved [5649/5649]

--2024-12-08 18:42:14--  https://raw.githubusercontent.com/martin-fabbri/recsys-pipeline/refs/heads/main/tiktok-pipeline/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiti

In [8]:
import pandas as pd
import numpy as np
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from features import generate_users
from features import generate_video_content
from features import generate_interactions

## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [9]:
user_data = generate_users(25_000)

data_users_df = pd.DataFrame(user_data)
data_users_df.head()

,user_id,gender,age,country
0,UA187I,Other,12,Samoa
1,PB710U,Male,28,Australia
2,BL370I,Other,37,Lebanon
3,UY482L,Other,19,French Southern Territories
4,QC514U,Other,62,Germany


### <span style='color:#ff5f27'> 🎥 Content Data Generation</span>


In [10]:
# Generate data for 25_000 videos
video_data = generate_video_content(25_000, historical=True)

data_video_df = pd.DataFrame(video_data)
data_video_df.head()

,video_id,category,views,likes,video_length,upload_date
0,5ZS32B,Sports,153687,67872,21,2023-10-05
1,2LV41T,Technology,174731,72174,17,2024-07-11
2,9LA68O,Education,74179,51660,67,2023-06-20
3,7BJ39X,Lifestyle,33326,32540,70,2023-08-17
4,1EL90U,Sports,150922,27428,137,2024-11-20


### <span style='color:#ff5f27'> 🔗 Interactions Generation</span>


In [11]:
num_interactions = 1_000_000

# Generate interactions
interactions = generate_interactions(num_interactions, user_data, video_data)

data_interactions_df = pd.DataFrame(interactions)
data_interactions_df.head()

,interaction_id,user_id,video_id,interaction_type,watch_time
0,0582-22-5482,IH718L,0IO90J,share,40
1,3736-68-1889,FT925D,5AJ06H,share,123
2,6501-54-3739,SI438Z,2DC62M,skip,177
3,2281-78-0798,UQ067K,4CH61Y,skip,76
4,2297-17-5006,TW474P,8PY65L,skip,178


## <span style="color:#ff5f27">👮🏻‍♂️ Great Expectations </span>

In [12]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_users_df = ge.from_pandas(data_users_df)

# Initialize the expectation suite
expectation_suite_users = ge_users_df.get_expectation_suite()
expectation_suite_users.expectation_suite_name = "user_data_suite"

# Expectation: Age should be between 12 and 120
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "age", "min_value": 12, "max_value": 120}
    )
)

# Expectations: Columns should not have null values
for column in ge_users_df.columns:
    expectation_suite_users.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Gender should only contain specific values
expectation_suite_users.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={"column": "gender", "value_set": ["Male", "Female", "Other"]}
    )
)

{"expectation_type": "expect_column_distinct_values_to_be_in_set", "kwargs": {"column": "gender", "value_set": ["Male", "Female", "Other"]}, "meta": {}}

In [13]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_video_df = ge.from_pandas(data_video_df)

# Initialize the expectation suite
expectation_suite_videos = ge_video_df.get_expectation_suite()
expectation_suite_videos.expectation_suite_name = "video_data_suite"

# Expectation: Views, Likes, and Video Length should be non-negative
for column in ["views", "likes", "video_length"]:
    expectation_suite_videos.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_between",
            kwargs={"column": column, "min_value": 0, "max_value": None}
        )
    )

# Expectation: Valid date format for upload_date
expectation_suite_videos.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_dateutil_parseable",
        kwargs={"column": "upload_date"}
    )
)

{"expectation_type": "expect_column_values_to_be_dateutil_parseable", "kwargs": {"column": "upload_date"}, "meta": {}}

In [ ]:
# Create a Great Expectations DataFrame from the pandas DataFrame
ge_interactions_df = ge.from_pandas(data_interactions_df)

# Initialize the expectation suite
expectation_suite_interactions = ge_interactions_df.get_expectation_suite()
expectation_suite_interactions.expectation_suite_name = "interactions_data_suite"

# Expectations: Non-null values in all columns
for column in ge_interactions_df.columns:
    expectation_suite_interactions.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_not_be_null",
            kwargs={"column": column}
        )
    )

# Expectation: Validate interaction types
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "interaction_type",
            "value_set": ['like', 'dislike', 'view', 'comment', 'share', 'skip']
        }
    )
)

# Expectation: Positive watch time
expectation_suite_interactions.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={"column": "watch_time", "min_value": 0}
    )
)

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27">🪄 Feature Group Creation </span>


In [ ]:
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Users data.",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_users,
)

users_fg.insert(data_users_df)
print('Done ✅')

In [ ]:
videos_fg = fs.get_or_create_feature_group(
    name="videos",
    description="Videos data.",
    version=1,
    primary_key=["video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_videos,
)

videos_fg.insert(data_video_df)
print('Done ✅')

In [ ]:
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    description="Interactions data.",
    version=1,
    primary_key=["interaction_id", "user_id", "video_id"],
    online_enabled=True,
    expectation_suite=expectation_suite_interactions,
)

interactions_fg.insert(data_interactions_df)
print('Done ✅')

## <span style="color:#ff5f27">🪄 Ranking Feature Group </span>


In [ ]:
video_interactions_df = pd.merge(
    data_interactions_df,
    data_video_df,
    on='video_id',
    how='inner',
)

ranking_df = pd.merge(
    video_interactions_df,
    data_users_df,
    on='user_id',
    how='inner',
)

ranking_df['label'] = np.where(
    ranking_df.interaction_type.isin(['view', 'like', 'share', 'comment']),
    1,
    0,
)

ranking_df.drop(
    ['interaction_id', 'interaction_type', 'watch_time'],
    axis=1,
    inplace=True,
)

ranking_df.head()

In [ ]:
ranking_fg = fs.get_or_create_feature_group(
    name="ranking",
    description="Ranking Data.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg.insert(ranking_df)
print('Done ✅')

---